# 🎲 Simulations Monte Carlo avec Heston

## Introduction

La méthode Monte Carlo est au cœur de la finance quantitative. Elle consiste à générer des milliers (voire millions) de scénarios aléatoires pour estimer des probabilités et des statistiques.

### 🎯 Objectifs
1. Générer des milliers de trajectoires Heston
2. Calculer des statistiques (mean, percentiles, etc.)
3. Estimer des probabilités (P(prix > seuil))
4. Créer des visualisations professionnelles

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pandas as pd
from tqdm import tqdm  # Barre de progression

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")
%matplotlib inline

In [ ]:
# Réutilisons notre classe Heston du notebook précédent
class HestonModel:
    def __init__(self, S0, V0, mu, kappa, theta, sigma_v, rho):
        self.S0 = S0
        self.V0 = V0
        self.mu = mu
        self.kappa = kappa
        self.theta = theta
        self.sigma_v = sigma_v
        self.rho = rho
    
    def simulate(self, T=1.0, N=1000, n_paths=1, seed=None):
        if seed is not None:
            np.random.seed(seed)
        
        dt = T / N
        sqrt_dt = np.sqrt(dt)
        t = np.linspace(0, T, N+1)
        
        S = np.zeros((N+1, n_paths))
        V = np.zeros((N+1, n_paths))
        S[0] = self.S0
        V[0] = self.V0
        
        rho_comp = np.sqrt(1 - self.rho**2)
        
        for i in range(N):
            Z1 = np.random.standard_normal(n_paths)
            Z2 = np.random.standard_normal(n_paths)
            
            dW_S = sqrt_dt * Z1
            dW_v = sqrt_dt * (self.rho * Z1 + rho_comp * Z2)
            
            V_current = np.maximum(V[i], 0)
            sqrt_V = np.sqrt(V_current)
            
            V[i+1] = V[i] + self.kappa * (self.theta - V_current) * dt \
                     + self.sigma_v * sqrt_V * dW_v
            V[i+1] = np.maximum(V[i+1], 0)
            
            S[i+1] = S[i] * np.exp((self.mu - 0.5*V_current) * dt + sqrt_V * dW_S)
        
        return t, S, V

## 1. Cas d'usage : Prédiction Bitcoin sur 30 jours

Imaginons que Bitcoin est à 90,000 USD et que nous voulons estimer sa distribution de prix dans 30 jours.

In [ ]:
# Paramètres réalistes pour Bitcoin
btc_model = HestonModel(
    S0=90000,         # Prix actuel
    V0=0.25,          # Variance initiale (50% vol annuelle)
    mu=0.20,          # 20% drift annuel (bullish)
    kappa=3.0,        # Retour rapide à la moyenne
    theta=0.25,       # Variance long terme
    sigma_v=0.6,      # Vol of vol élevé (crypto)
    rho=-0.75         # Corrélation négative forte
)

# Simulons 30 jours avec 50,000 scénarios
T_days = 30
T_years = T_days / 365
N_steps = T_days  # 1 step = 1 jour
n_simulations = 50000

print("🚀 Lancement de la simulation Monte Carlo...")
print(f"   Paramètres: {n_simulations:,} trajectoires sur {T_days} jours")
print()

t, S_paths, V_paths = btc_model.simulate(
    T=T_years, 
    N=N_steps, 
    n_paths=n_simulations,
    seed=42
)

print("✅ Simulation terminée!")
print(f"   Taille mémoire: S_paths = {S_paths.nbytes / 1e6:.1f} MB")

## 2. Visualisation des trajectoires

In [ ]:
# Visualiser un échantillon de trajectoires
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))

# Prix: afficher 100 trajectoires
n_display = 100
for i in range(n_display):
    ax1.plot(t * 365, S_paths[:, i], alpha=0.1, linewidth=0.5, color='blue')

# Ajouter percentiles
p5 = np.percentile(S_paths, 5, axis=1)
p50 = np.percentile(S_paths, 50, axis=1)
p95 = np.percentile(S_paths, 95, axis=1)

ax1.plot(t * 365, p50, 'r-', linewidth=3, label='Médiane (P50)')
ax1.plot(t * 365, p5, 'g--', linewidth=2, label='P5')
ax1.plot(t * 365, p95, 'orange', linestyle='--', linewidth=2, label='P95')
ax1.fill_between(t * 365, p5, p95, alpha=0.2, color='gray', label='Intervalle 90%')
ax1.axhline(y=90000, color='black', linestyle=':', linewidth=2, label='Prix initial')

ax1.set_title(f'Bitcoin - {n_simulations:,} Trajectoires Monte Carlo sur {T_days} jours', 
             fontsize=16, fontweight='bold')
ax1.set_xlabel('Jours')
ax1.set_ylabel('Prix (USD)')
ax1.legend(loc='best', fontsize=11)
ax1.grid(True, alpha=0.3)

# Variance
for i in range(n_display):
    ax2.plot(t * 365, V_paths[:, i], alpha=0.1, linewidth=0.5, color='purple')

v_median = np.median(V_paths, axis=1)
ax2.plot(t * 365, v_median, 'r-', linewidth=3, label='Médiane')
ax2.axhline(y=btc_model.theta, color='green', linestyle='--', linewidth=2, label='θ (long terme)')
ax2.set_title('Variance v(t)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Jours')
ax2.set_ylabel('Variance')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Analyse statistique complète

In [ ]:
# Prix finaux (jour 30)
prix_finaux = S_paths[-1, :]

# Calcul des statistiques
stats_dict = {
    'Prix initial': btc_model.S0,
    'Moyenne': np.mean(prix_finaux),
    'Médiane': np.median(prix_finaux),
    'Écart-type': np.std(prix_finaux),
    'Min': np.min(prix_finaux),
    'Max': np.max(prix_finaux),
    'P5': np.percentile(prix_finaux, 5),
    'P25': np.percentile(prix_finaux, 25),
    'P50': np.percentile(prix_finaux, 50),
    'P75': np.percentile(prix_finaux, 75),
    'P95': np.percentile(prix_finaux, 95),
    'Skewness': stats.skew(prix_finaux),
    'Kurtosis': stats.kurtosis(prix_finaux)
}

# Affichage professionnel
df_stats = pd.DataFrame(list(stats_dict.items()), columns=['Statistique', 'Valeur'])
df_stats['Valeur'] = df_stats['Valeur'].apply(lambda x: f"{x:,.2f}" if abs(x) > 10 else f"{x:.4f}")

print("\n" + "="*60)
print("📊 STATISTIQUES DE LA DISTRIBUTION DES PRIX (Jour 30)")
print("="*60)
print(df_stats.to_string(index=False))
print("="*60)

## 4. Distribution des prix finaux - Visualisations avancées

In [ ]:
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Histogramme principal
ax1 = fig.add_subplot(gs[0, :])
counts, bins, patches = ax1.hist(prix_finaux, bins=100, alpha=0.7, 
                                  edgecolor='black', density=True)

# Colorer par zones
for i, patch in enumerate(patches):
    if bins[i] < stats_dict['P5']:
        patch.set_facecolor('red')
    elif bins[i] > stats_dict['P95']:
        patch.set_facecolor('green')
    else:
        patch.set_facecolor('skyblue')

ax1.axvline(btc_model.S0, color='black', linestyle=':', linewidth=3, label='Prix initial')
ax1.axvline(np.mean(prix_finaux), color='blue', linestyle='--', linewidth=2, label='Moyenne')
ax1.axvline(np.median(prix_finaux), color='red', linestyle='--', linewidth=2, label='Médiane')
ax1.axvline(stats_dict['P5'], color='darkred', linestyle='--', linewidth=1.5, alpha=0.7, label='P5')
ax1.axvline(stats_dict['P95'], color='darkgreen', linestyle='--', linewidth=1.5, alpha=0.7, label='P95')

ax1.set_title(f'Distribution des Prix Bitcoin après {T_days} jours ({n_simulations:,} simulations)',
             fontsize=16, fontweight='bold')
ax1.set_xlabel('Prix (USD)', fontsize=12)
ax1.set_ylabel('Densité', fontsize=12)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 2. Box plot
ax2 = fig.add_subplot(gs[1, 0])
bp = ax2.boxplot(prix_finaux, vert=True, patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
ax2.set_ylabel('Prix (USD)')
ax2.set_title('Box Plot', fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. Violin plot
ax3 = fig.add_subplot(gs[1, 1])
parts = ax3.violinplot([prix_finaux], vert=True, showmeans=True, showmedians=True)
ax3.set_ylabel('Prix (USD)')
ax3.set_title('Violin Plot', fontweight='bold')
ax3.grid(True, alpha=0.3)

# 4. Percentiles visuels
ax4 = fig.add_subplot(gs[1, 2])
percentiles = [5, 10, 25, 50, 75, 90, 95]
perc_values = np.percentile(prix_finaux, percentiles)
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(percentiles)))
bars = ax4.barh(percentiles, perc_values, color=colors, edgecolor='black')
ax4.axvline(btc_model.S0, color='black', linestyle=':', linewidth=2, label='Initial')
ax4.set_xlabel('Prix (USD)')
ax4.set_ylabel('Percentile')
ax4.set_title('Percentiles', fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='x')

# 5. Cumulative Distribution Function
ax5 = fig.add_subplot(gs[2, :])
sorted_prices = np.sort(prix_finaux)
cdf = np.arange(1, len(sorted_prices)+1) / len(sorted_prices)
ax5.plot(sorted_prices, cdf * 100, linewidth=2, color='navy')
ax5.axvline(btc_model.S0, color='red', linestyle='--', linewidth=2, label='Prix initial')
ax5.axhline(50, color='gray', linestyle=':', alpha=0.5)
ax5.set_title('Fonction de Répartition Cumulative (CDF)', fontsize=14, fontweight='bold')
ax5.set_xlabel('Prix (USD)')
ax5.set_ylabel('Probabilité Cumulative (%)')
ax5.grid(True, alpha=0.3)
ax5.legend()

plt.show()

## 5. Calcul de probabilités - Questions pratiques

In [ ]:
# Questions qu'un trader/investisseur pourrait poser
seuils = [80000, 90000, 100000, 110000, 120000]

print("\n" + "="*70)
print("🎯 PROBABILITÉS - Questions Pratiques")
print("="*70)
print()

results = []
for seuil in seuils:
    prob_above = (prix_finaux > seuil).sum() / len(prix_finaux) * 100
    prob_below = 100 - prob_above
    
    # Intervalle de confiance (approximation binomiale)
    p = prob_above / 100
    n = len(prix_finaux)
    se = np.sqrt(p * (1-p) / n)
    ci_lower = max(0, (p - 1.96*se) * 100)
    ci_upper = min(100, (p + 1.96*se) * 100)
    
    results.append({
        'Seuil (USD)': f"{seuil:,}",
        'P(Prix > Seuil)': f"{prob_above:.2f}%",
        'IC 95%': f"[{ci_lower:.2f}%, {ci_upper:.2f}%]",
        'P(Prix < Seuil)': f"{prob_below:.2f}%"
    })

df_probs = pd.DataFrame(results)
print(df_probs.to_string(index=False))
print("\n" + "="*70)
print()

# Buckets (intervalles)
print("\n📦 PROBABILITÉS PAR INTERVALLES (Buckets)")
print("="*70)
buckets = [(0, 80000), (80000, 90000), (90000, 100000), 
           (100000, 110000), (110000, 150000)]

bucket_results = []
for low, high in buckets:
    prob = ((prix_finaux >= low) & (prix_finaux < high)).sum() / len(prix_finaux) * 100
    bucket_results.append({
        'Intervalle': f"[{low:,}, {high:,})",
        'Probabilité': f"{prob:.2f}%"
    })

df_buckets = pd.DataFrame(bucket_results)
print(df_buckets.to_string(index=False))
print("="*70)

## 6. Analyse de sensibilité - Impact du temps

In [ ]:
# Comment la distribution évolue-t-elle dans le temps ?
jours_analyse = [7, 14, 21, 30]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for idx, jour in enumerate(jours_analyse):
    prix_jour = S_paths[jour, :]
    
    axes[idx].hist(prix_jour, bins=60, alpha=0.7, edgecolor='black', density=True)
    axes[idx].axvline(btc_model.S0, color='red', linestyle=':', linewidth=2, label='Initial')
    axes[idx].axvline(np.mean(prix_jour), color='blue', linestyle='--', linewidth=2, label='Moyenne')
    axes[idx].axvline(np.median(prix_jour), color='green', linestyle='--', linewidth=2, label='Médiane')
    
    axes[idx].set_title(f'Distribution après {jour} jours', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Prix (USD)')
    axes[idx].set_ylabel('Densité')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)
    
    # Stats
    mean = np.mean(prix_jour)
    std = np.std(prix_jour)
    stats_text = f"μ={mean:,.0f}\nσ={std:,.0f}\nCV={std/mean:.2%}"
    axes[idx].text(0.02, 0.98, stats_text, transform=axes[idx].transAxes,
                  fontsize=10, verticalalignment='top',
                  bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

plt.suptitle('Évolution temporelle de la distribution', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("📈 Observation: L'incertitude (écart-type) augmente avec le temps")

## 🎯 Résumé

### Ce que nous avons appris :

1. **Monte Carlo** : méthode puissante pour estimer des probabilités complexes
2. **Statistiques descriptives** : mean, median, percentiles, skewness, kurtosis
3. **Probabilités conditionnelles** : P(Prix > seuil), buckets
4. **Intervalles de confiance** : mesurer la précision de nos estimations
5. **Analyse temporelle** : évolution de la distribution

### 🔑 Points clés :
- Plus de simulations → meilleure précision
- La distribution s'élargit avec le temps (incertitude croissante)
- Heston capture l'asymétrie (skewness) des rendements réels

### 📖 Prochain notebook : Génération de rapports

Dans le dernier notebook, nous allons :
- Créer des visualisations professionnelles comme dans Heston.v2
- Générer un rapport HTML complet
- Combiner toutes les analyses

---

**Continuez vers 05_Generation_Rapports.ipynb** 🚀